In [25]:
# 2025/12/8
# zhangzhong
# zhangzhong
# https://docs.langchain.com/oss/python/integrations/vectorstores/milvus

In [26]:
import os
api_key: str = os.environ["BIGMODEL_API_KEY"]

In [27]:
## Initialization
# https://docs.bigmodel.cn/cn/guide/develop/langchain/introduction
# 看智谱的文档，直接用openai的库就行，zhipu的api和openai的api是兼容的

from langchain_openai import OpenAIEmbeddings

# 卧槽！真的兼容！
embeddings = OpenAIEmbeddings(model="embedding-3", base_url="https://open.bigmodel.cn/api/paas/v4", api_key=api_key)


In [28]:
# Milvus Lite
# The easiest way to prototype is to use Milvus Lite, where everything is stored in a local vector database file. Only the Flat index can be used.

from langchain_milvus import Milvus

# URI = "./milvus_example.db"
# we could use milvus standalone
URI = "http://localhost:19530"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

In [29]:
# or use milvus stand alone
# https://docs.langchain.com/oss/python/integrations/vectorstores/milvus#milvus-server

In [30]:
## Store unrelated doc in different colections

from langchain_core.documents import Document

vector_store_saved = Milvus.from_documents(
    [Document(page_content="foo!")],
    embeddings,
    collection_name="langchain_example",
    connection_args={"uri": URI},
)

# And here is how you retrieve that stored collection:
vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="langchain_example",
)

In [31]:
## Manage vector store
# Once you have created your vector store, we can interact with it by adding and deleting different items.


from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['def79618-bad0-4fd9-9796-fc1d7c933432',
 'efc499a8-1003-4108-bb00-e3126d02576a',
 'a118bbbd-b90e-4828-bd32-72af819f20d8',
 '8912e4a8-339c-4be9-844c-d1f08cad15cf',
 'eb0ae1e3-1661-4404-befc-f9f5762f3976',
 'bdab6560-e6d7-485e-ad2a-f2c9ad17b3b5',
 '3c3e0b56-658a-43d7-b3d3-28f3b1633329',
 '59eff57f-da51-4775-838c-72bab60904a0',
 '8bbd7c99-90ff-4be8-81cc-f2b415b1a7ea',
 '0b24c65b-874b-459d-a73d-be2306397358']

In [32]:
## Delete items from vector store

vector_store.delete(ids=[uuids[-1]])


True

In [33]:
## Query vector store

# similarity search
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    expr='source == "tweet"',
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet', 'pk': '59eff57f-da51-4775-838c-72bab60904a0'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet', 'pk': 'a118bbbd-b90e-4828-bd32-72af819f20d8'}]


In [34]:
# Similarity search with score
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, expr='source == "news"'
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.888516] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news', 'pk': 'efc499a8-1003-4108-bb00-e3126d02576a'}]


In [35]:
# vector store as retriever
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
# TIPS: 这里写filter回报错，可能是版本不兼容
retriever.invoke("Stealing from the bank is a crime")

[Document(metadata={'source': 'news', 'pk': '8912e4a8-339c-4be9-844c-d1f08cad15cf'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [ ]:
## Hybrid Search
# The most common hybrid search scenario is the dense + sparse hybrid search, 
# where candidates are retrieved using both semantic vector similarity and precise keyword matching. 
# Results from these methods are merged, reranked, and passed to an LLM to generate the final answer


from langchain_milvus import BM25BuiltInFunction, Milvus
from langchain_openai import OpenAIEmbeddings

vectorstore = Milvus.from_documents(
    documents=documents,
    embedding=embeddings,
    # When you use BM25BuiltInFunction, please note that the full-text search is available in Milvus Standalone and Milvus Distributed, but not in Milvus Lite,
    # BM25BuiltInFunction does not require the client to pass corpus or training, all are automatically processed at the Milvus server’s end, so users do not need to care about any vocabulary and corpu
    builtin_function=BM25BuiltInFunction(),
    # `dense` is for OpenAI embeddings, `sparse` is the output field of BM25 function
    vector_field=["dense", "sparse"],
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

In [37]:
## Rerank
# After the first stage of retrieval, we need to rerank the candidates to get a better result

query = "What are the novels Lila has written and what are their contents?"

vectorstore.similarity_search(
    query, k=1, ranker_type="weighted", ranker_params={"weights": [0.6, 0.4]}
)

[Document(metadata={'source': 'tweet', 'pk': 462705275283988660}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [ ]:
## Per-User Retrieval
# When building a retrieval app, you often have to build it with multiple users in mind. This means that you may be storing data not just for one user, but for many different users, and they should not be able to see each other’s data.
# Milvus recommends using partition_key to implement [multi-tenanc]

from langchain_core.documents import Document

docs = [
    Document(page_content="i worked at kensho", metadata={"namespace": "harrison"}),
    Document(page_content="i worked at facebook", metadata={"namespace": "ankush"}),
]
vectorstore = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"uri": URI},
    drop_old=True,
    # # Milvus recommends using partition_key to implement [multi-tenanc]
    partition_key_field="namespace",  # Use the "namespace" field as the partition key
)

# search
# To conduct a search using the partition key, you should include either of the following in the boolean expression of the search request:
# search_kwargs={"expr": '<partition_key> == "xxxx"'}
# search_kwargs={"expr": '<partition_key> == in ["xxx", "xxx"]'}
# 